In [1]:
import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from preprocess_utility import *
from data_manager import *
from metadata import *

Setting environment for Gordon


No vtk


# Generate score volumes

In [ ]:
for stack in all_stacks:
    
# for stack in ['MD603']:
        
    first_sec, last_sec = metadata_cache['section_limits'][stack]

    #################################

    t = time.time()
    sys.stderr.write('running svm classifier ...')

    exclude_nodes = [33]

    run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/svm_predict.py',
                    'stack': stack},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

#     #################################

    t = time.time()
    sys.stderr.write('interpolating scoremaps ...')

    exclude_nodes = [33]

    run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/interpolate_scoremaps_v2.py',
                    'stack': stack},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # ~240 seconds 

    #################################

    t = time.time()
    sys.stderr.write('visualize scoremaps ...')

    add_annotation = False

    exclude_nodes = [33]
    # first_sec, last_sec = DataManager.load_cropbox(stack)[4:]

    run_distributed4(command='%(script_path)s %(stack)s -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/visualize_scoremaps_v2.py',
                    'stack': stack,
                    'add_annotation': '-a' if add_annotation else ''},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 464 seconds / stack

    #################################

    paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                        'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
    singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
    structures = paired_structures + singular_structures

    #################################

    t = time.time()
    sys.stderr.write('constructing score volumes ...')

    exclude_nodes = [33]

    run_distributed4(command='%(script_path)s %(stack)s %%(label)s' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct') + '/construct_score_volume_v2.py',
                    'stack': stack},
                    kwargs_list=dict(label=structures),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t))

    #################################

    downscale_factor = 32

    #################################

    print stack

    volume_allLabels = {}

    for name_u in structures:
        volume = DataManager.load_score_volume(stack, label=name_u, downscale=downscale_factor, train_sample_scheme=1)
        volume_allLabels[name_u] = volume
#         del volume

    t1 = time.time()

    gradient_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack, 'score_volume_gradients'))

    for name_u in structures:

        t = time.time()

        gy_gx_gz = np.gradient(volume_allLabels[name_u].astype(np.float32), 3, 3, 3) 
        # 3.3 second - re-computing is much faster than loading
        # .astype(np.float32) is important; 
        # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
        # 20s (float32) vs. 2s (float16)

        sys.stderr.write('Gradient %s: %f seconds\n' % (name_u, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_gx_gz[0], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gy.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[1], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gx.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[2], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gz.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name_u, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

interpolating scoremaps ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


# Global align

In [4]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/align_subject_brain_to_atlas_v2.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # 526 seconds

align all subjects to atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 526.264969 seconds


In [6]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 270.454302 seconds


In [8]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/visualize_aligned_brains_v2.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 625 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 624.721753 seconds


# Local align

In [10]:
t = time.time()
sys.stderr.write('LOCAL align ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_atlas_structure_to_subject_v2.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 6807 seconds

LOCAL align ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 6807.340681 seconds


In [11]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains_global_to_local.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 257.636058 seconds


In [12]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/visualize_aligned_brains_v2_local.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 625 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 190.024241 seconds


In [13]:
# Transform locally transformed volumes back to atlas space

t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains_reverse_global.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 220.844733 seconds
